In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import pandas as pd
import medmnist
from medmnist import INFO, Evaluator
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import manifold
from scipy import stats
from PIL import Image
import pdb
import math

/home/grads/a/akshatha.mohan/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/grads/a/akshatha.mohan/anaconda3/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Example usage:
image = torch.tensor([[9, 11, 8, 20], [18, 18, 17, 3], [10, 7, 18, 16], [18, 17, 22, 13]], dtype=torch.float32)
image = image.unsqueeze(0).unsqueeze(0)
image


tensor([[[[ 9., 11.,  8., 20.],
          [18., 18., 17.,  3.],
          [10.,  7., 18., 16.],
          [18., 17., 22., 13.]]]])

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import math
import pdb

class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        image_scaled = image * 255
        windows_horizontal = []
        Lr_all_horizontal = []

        for i in range(image_scaled.size(2) - self.window_size + 1):
            for j in range(image_scaled.size(3) - self.window_size + 1):
                window = image_scaled[:, :, i:i+self.window_size, j:j+self.window_size]
                windows_horizontal.append(window)

                max_pool = nn.MaxPool2d(kernel_size=self.window_size)
                max_pool_output = max_pool(window)
                min_pool_output = -max_pool(-window)

                nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
                Mr = torch.sum(nr)
                Q_mr = nr / (self.window_size - self.r + 1)
                L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2

        return L_r

# Example usage:
# Create an instance of the CustomPoolingLayer
custom_pooling_layer = CustomPoolingLayer(r=3, window_size=3)
result = custom_pooling_layer(image)
print(result)


tensor([[[[0.0006]]]])


In [8]:
import torch
import torch.nn as nn
import math

class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        windows_horizontal = image.unfold(2, self.window_size, 1).unfold(3, self.window_size, 1)
        windows_horizontal = windows_horizontal.squeeze(0).squeeze(0)
        print(windows_horizontal.shape)
        max_pool = nn.MaxPool2d(kernel_size=self.window_size)
        max_pool_output = max_pool(windows_horizontal)
        min_pool_output = -max_pool(-windows_horizontal)

        nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
        Mr = torch.sum(nr)
        Q_mr = nr / (self.window_size - self.r + 1)
        L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2

        return L_r

# Example usage:
# Create an instance of the CustomPoolingLayer
custom_pooling_layer = CustomPoolingLayer(r=3, window_size=3)

# Assuming 'your_image_tensor' is your input tensor
image = torch.tensor([[9, 11, 8, 20], [18, 18, 17, 3], [10, 7, 18, 16], [18, 17, 22, 13]], dtype=torch.float32)
image = image.unsqueeze(0).unsqueeze(0)

# Get the result using the custom pooling layer
result = custom_pooling_layer(image)
print(result)


torch.Size([2, 2, 3, 3])
tensor([[[[0.5000]],

         [[0.2000]]],


        [[[0.2500]],

         [[0.1667]]]])


Batch dimension stack

In [74]:
import torch
import torch.nn as nn

class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        batch_size, channels, image_height, image_width = image.size()

        # Initialize variables to accumulate results across batches
        all_L_r = []

        # Iterate over batches
        for batch in range(batch_size):
            # Extract the current batch's unfolded window
            windows_horizontal = image[batch].unfold(1, self.window_size, 1).unfold(2, self.window_size, 1)
            windows_horizontal = windows_horizontal.squeeze(0)

            # Perform operations independently for each window in the current batch
            max_pool = nn.MaxPool2d(kernel_size=self.window_size)
            max_pool_output = max_pool(windows_horizontal)
            min_pool_output = -max_pool(-windows_horizontal)

            nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
            Mr = torch.sum(nr)
            Q_mr = nr / (self.window_size - self.r + 1)
            L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2

            # Accumulate the result for the current batch
            all_L_r.append(L_r)

        # Concatenate results along the batch dimension
        all_L_r = torch.cat(all_L_r, dim=0)

        return all_L_r

# Example usage:
# Create an instance of the CustomPoolingLayer
custom_pooling_layer = CustomPoolingLayer(r=3, window_size=3)


image = torch.tensor([[9, 11, 8, 20], [18, 18, 17, 3], [10, 7, 18, 16], [18, 17, 22, 13]], dtype=torch.float32)
image = image.unsqueeze(0).unsqueeze(0)

# Get the result using the custom pooling layer
result = custom_pooling_layer(image)
print(result.shape)


torch.Size([2, 2, 1, 1])


Channel dimension stack

In [38]:
import torch
import torch.nn as nn

class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        # Scale the input tensor to the range [0, 255]
        image_scaled = image * 255.0

        windows_horizontal = image_scaled.unfold(2, self.window_size, 1).unfold(3, self.window_size, 1)
        windows_horizontal = windows_horizontal.squeeze(0).squeeze(0)

        max_pool = nn.MaxPool2d(kernel_size=self.window_size)
        max_pool_output = max_pool(windows_horizontal)
        min_pool_output = -max_pool(-windows_horizontal)


        nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
        Mr = torch.sum(nr)
        Q_mr = nr / (self.window_size - self.r + 1)
        L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2

        return L_r

# Example usage:
# Create an instance of the CustomPoolingLayer
custom_pooling_layer = CustomPoolingLayer(r=3, window_size=3)

# Assuming 'your_image_tensor' is your input tensor with shape (1, 1, 4, 4)
image = torch.rand((1, 1, 4, 4), dtype=torch.float32)

# Get the result using the custom pooling layer
result = custom_pooling_layer(image)
print(result.shape)


torch.Size([2, 2, 1, 1])


Normalized dimension stack

In [17]:
import torch
import torch.nn as nn

class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        image_scaled = image * 255
        batch_size, channels, image_height, image_width = image.size()

        # Initialize variables to accumulate results across batches
        all_L_r = []
        all_L_r_channel = []

        # Iterate over batches
        for batch in range(batch_size):
            # Iterate over channels
            channel_L_r = []
            for channel in range(channels):
                # Extract the current batch's unfolded window for the current channel
                windows_horizontal = image_scaled[batch, channel].unfold(0, self.window_size, 1).unfold(1, self.window_size, 1)
                windows_horizontal = windows_horizontal.squeeze(0)

                # Perform operations independently for each window in the current channel
                max_pool = nn.MaxPool2d(kernel_size=self.window_size)
                max_pool_output = max_pool(windows_horizontal)
                min_pool_output = -max_pool(-windows_horizontal)

                nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
                Mr = torch.sum(nr)
                Q_mr = nr / (self.window_size - self.r + 1)
                L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2
                # Accumulate the result for the current channel
                channel_L_r.append(L_r)

            # Concatenate results along the channel dimension for the current batch
            channel_L_r = torch.cat(channel_L_r, dim=0)
            all_L_r_channel.append(channel_L_r)

        # Concatenate results along the batch dimension
        all_L_r_batch = torch.cat(all_L_r_channel, dim=1)

        return all_L_r_batch

# Example usage:
# Create an instance of the CustomPoolingLayer
custom_pooling_layer = CustomPoolingLayer(r=3, window_size=3)

# Assuming 'your_image_tensor' is your input tensor with shape [128, 1, 13, 13] (128 batches, 1 channel, 13x13 image)
image = torch.rand((128, 3, 13, 13), dtype=torch.float32)


# Get the result using the custom pooling layer
result = custom_pooling_layer(image)
print(result)


tensor([[[[0.0208]],

         [[0.0145]],

         [[0.0145]],

         ...,

         [[0.0169]],

         [[0.0137]],

         [[0.0159]]],


        [[[0.0149]],

         [[0.0156]],

         [[0.0152]],

         ...,

         [[0.0169]],

         [[0.0137]],

         [[0.0141]]],


        [[[0.0128]],

         [[0.0167]],

         [[0.0141]],

         ...,

         [[0.0172]],

         [[0.0185]],

         [[0.0154]]],


        ...,


        [[[0.0137]],

         [[0.0135]],

         [[0.0145]],

         ...,

         [[0.0145]],

         [[0.0145]],

         [[0.0123]]],


        [[[0.0133]],

         [[0.0133]],

         [[0.0147]],

         ...,

         [[0.0145]],

         [[0.0145]],

         [[0.0123]]],


        [[[0.0137]],

         [[0.0137]],

         [[0.0175]],

         ...,

         [[0.0179]],

         [[0.0179]],

         [[0.0154]]]])


In [43]:
class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        image_scaled = image * 255
        batch_size, channels, image_height, image_width = image.size()

        # Initialize variables to accumulate results across batches
        all_L_r = []

        # Iterate over batches
        for batch in range(batch_size):
            # Iterate over channels
            channel_L_r = []
            for channel in range(channels):
                # Extract the current batch's unfolded window for the current channel
                windows_horizontal = image_scaled[batch, channel].unfold(0, self.window_size, 1).unfold(1, self.window_size, 1)
                windows_horizontal = windows_horizontal.squeeze(0)

                # Perform operations independently for each window in the current channel
                max_pool = nn.MaxPool2d(kernel_size=self.window_size)
                max_pool_output = max_pool(windows_horizontal)
                min_pool_output = -max_pool(-windows_horizontal)

                nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
                Mr = torch.sum(nr)
                Q_mr = nr / (self.window_size - self.r + 1)
                L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2

                # Accumulate the result for the current channel
                channel_L_r.append(L_r)

            # Stack results across channels for the current batch
            channel_L_r = torch.stack(channel_L_r, dim = 1)
            all_L_r.append(channel_L_r)

        # Stack results across batches
        all_L_r = torch.stack(all_L_r)

        return all_L_r
    

# Create an instance of the CustomPoolingLayer
custom_pooling_layer = CustomPoolingLayer(r=3, window_size=3)

# Assuming 'your_image_tensor' is your input tensor with shape [128, 1, 13, 13] (128 batches, 1 channel, 13x13 image)
image = torch.rand((1, 1, 4, 4), dtype=torch.float32)


# Get the result using the custom pooling layer
result = custom_pooling_layer(image)
print(result.shape)

torch.Size([2, 2, 3, 3])
torch.Size([1, 2, 1, 2, 1, 1])


In [10]:
class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        image_scaled = image * 255
        batch_size, channels, image_height, image_width = image.size()

        # Initialize variables to accumulate results across batches
        all_L_r = []

        # Iterate over batches
        for batch in range(batch_size):
            # Iterate over channels
            channel_L_r = []
            for channel in range(channels):
                # Extract the current batch's unfolded window for the current channel
                windows_horizontal = image_scaled[batch, channel].unfold(0, self.window_size, 1).unfold(1, self.window_size, 1)
                # Perform operations independently for each window in the current channel
                max_pool = nn.MaxPool2d(kernel_size=self.window_size)
                max_pool_output = max_pool(windows_horizontal)
                min_pool_output = -max_pool(-windows_horizontal)

                nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
                Mr = torch.sum(nr)
                Q_mr = nr / (self.window_size - self.r + 1)
                L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2
                L_r = L_r.squeeze(-1).squeeze(-1)
                channel_L_r.append(L_r)

            # Stack results along the channel dimension
            channel_L_r = torch.stack(channel_L_r, dim=0)
            all_L_r.append(channel_L_r)

        # Stack results along the batch dimension
        all_L_r = torch.stack(all_L_r, dim=0)
        print(all_L_r.shape)

        return all_L_r

# Example usage
custom_pooling = CustomPoolingLayer(r=3, window_size=3)
input_data = torch.rand(1, 3, 4, 4)
output = custom_pooling(input_data)


torch.Size([1, 3, 2, 2])


In [19]:
class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        m = nn.Softmax2d()
        image = m(image) * 255
        batch_size, channels, image_height, image_width = image.size()

        # Perform operations independently for each window in the current channel
        max_pool = nn.MaxPool2d(kernel_size=self.window_size, stride=1)
        max_pool_output = max_pool(image)
        min_pool_output = -max_pool(-image)

        nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
        Mr = torch.sum(nr)
        Q_mr = nr / (self.window_size - self.r + 1)
        L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2
        L_r = L_r.squeeze(-1).squeeze(-1)

        return L_r

image = torch.rand(3, 3, 4, 4)
custom_pool = CustomPoolingLayer(r=3, window_size=3)
result = custom_pool(image)

print(result.shape)

torch.Size([3, 3, 2, 2])


In [1]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import transforms
import pdb
import matplotlib.pyplot as plt
import math
import torch.nn.functional as F


class Global_Lacunarity(nn.Module):
    def __init__(self, dim=2, eps = 10E-6, scales = None, kernel = None, stride = None, padding = None):


        # inherit nn.module
        super(Global_Lacunarity, self).__init__()

        # define layer properties
        self.dim = dim
        self.eps = eps
        self.kernel = kernel
        self.stride = stride
        self.padding = padding
        self.scales = scales
        self.conv1x1 = nn.Conv2d(len(self.scales) * 3, 3, kernel_size=1)

        #For each data type, apply two 1x1 convolutions, 1) to learn bin center (bias)
        # and 2) to learn bin width
        # Time series/ signal Data
        if self.kernel is None:
            if self.dim == 1:
                self.gap_layer = nn.AdaptiveAvgPool1d(1)
            
            # Image Data
            elif self.dim == 2:
                self.gap_layer = nn.AdaptiveAvgPool2d(1)
            
            # Spatial/Temporal or Volumetric Data
            elif self.dim == 3:
                self.gap_layer = nn.AdaptiveAvgPool3d(1)
             
            else:
                raise RuntimeError('Invalid dimension for global lacunarity layer')
        else:
            if self.dim == 1:
                self.gap_layer = nn.AvgPool1d((kernel[0]), stride=stride[0], padding=(0))
            
            # Image Data
            elif self.dim == 2:
                self.gap_layer = nn.AvgPool2d((kernel[0], kernel[1]), stride=(stride[0], stride[1]), padding=(0, 0))
            
            # Spatial/Temporal or Volumetric Data
            elif self.dim == 3:
                self.gap_layer = nn.AvgPool3d((kernel[0], kernel[1], kernel[2]), stride=(stride[0], stride[1], stride[2]), padding=(0, 0, 0))
             
            else:
                raise RuntimeError('Invalid dimension for global lacunarity layer')

        
    def forward(self,x):
        #Compute squared tensor
        #pdb.set_trace()
        lacunarity_values = []
        for scale in self.scales:
            scaled_x = x * scale
            squared_x_tensor = scaled_x ** 2

            #Get number of samples
            n_pts = np.prod(np.asarray(scaled_x.shape[-2:]))
            if (self.kernel == None):
                n_pts = np.prod(np.asarray(scaled_x.shape[-2:]))


            #Compute numerator (n * sum of squared pixels) and denominator (squared sum of pixels)
            L_numerator = ((n_pts)**2) * (self.gap_layer(squared_x_tensor))
            L_denominator = (n_pts * self.gap_layer(scaled_x))**2

            #Lacunarity is L_numerator / L_denominator - 1
            L_r = (L_numerator / (L_denominator + self.eps)) - 1
            lambda_param = 0.5 #boxcox transformation
            y = (torch.pow(L_r.abs() + 1, lambda_param) - 1) / lambda_param

            lacunarity_values.append(y)

        result = torch.cat(lacunarity_values, dim=1)
        reduced_output = self.conv1x1(result)
        return reduced_output



class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3, eps = 10E-6):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size
        self.normalize = nn.Softmax2d()
        self.r_values = [0.015, 0.0625, 0.5, 0.25, 0.125, 0.2, 0.4, 0.3, 0.75, 0.6, 0.9, 0.8, 1]
        self.num_output_channels = 3
        self.eps = eps
        self.conv1x1 = nn.Conv2d(len(self.r_values) * 3, self.num_output_channels, kernel_size=1)


    def forward(self, image):
        image = self.normalize(image) * 255
        L_r_all = []

        # Perform operations independently for each window in the current channel
        for r in self.r_values:
            max_pool = nn.MaxPool2d(kernel_size=self.window_size, stride=1)
            max_pool_output = max_pool(image)
            min_pool_output = -max_pool(-image)

            nr = torch.ceil(max_pool_output / r) - torch.ceil(min_pool_output / r) - 1
            Mr = torch.sum(nr)
            Q_mr = nr / (self.window_size - r + 1)
            L_r = (Mr**2) * Q_mr / (Mr * Q_mr + self.eps)**2
            L_r = L_r.squeeze(-1).squeeze(-1)
            L_r_all.append(L_r)
        channel_L_r = torch.cat(L_r_all, dim=1)
        reduced_output = self.conv1x1(channel_L_r)

        return reduced_output




/home/grads/a/akshatha.mohan/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/grads/a/akshatha.mohan/anaconda3/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
